Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подключение библиотек 

In [29]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import re 

from pymystem3 import Mystem

from sklearn.metrics import f1_score, make_scorer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier

## Подготовка

### Первичный осмотр

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


#### Вывод

Данные не содержат проблем 

### Деление данные на выборки

In [5]:
train, test = train_test_split(data, test_size=0.1, random_state=12345) 

### Лемматизация

In [6]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = text.split() 
    return ' '.join(text)

In [7]:
m = Mystem()
def lemmatize(text):
    text = clear_text(text)
    lemm_list = m.lemmatize(text)
    lemm_text = " ".join(lemm_list)
        
    return lemm_text

In [8]:
train.loc[:]['text'] = train['text'].apply(lemmatize)

In [9]:
train.head()

,text,toxic
51303,There is something wrong with you ...,0
22874,Amyas Godfrey The page has come un...,0
52848,I have now reported your fourth re...,0
130358,As I said earlier I respect the ...,0
138967,It is called a war in Poland It ...,0


### Создание признаков для обучения 

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
count_tf_idf.fit(train['text'])

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [18]:
features = count_tf_idf.transform(train['text'])

## Обучение

In [20]:
grid_report = pd.DataFrame(columns=['model', 'rmse', 'params'])

In [25]:
f1 = make_scorer(f1_score)

In [38]:
def my_GridSearchCV(mode, parametrs, train_x, train_y):
    
    grid = GridSearchCV(model(), parametrs, cv=5, scoring=f1)
    grid.fit(train_x, train_y)
    
    row = [model.__name__, grid.best_score_, grid.best_params_]
    return pd.DataFrame([row], columns=['model', 'rmse', 'params'])

### Простой классификатор   

In [39]:
parametrs = { 'strategy' : ('stratified', 'most_frequent', 'prior', 'uniform')}

In [40]:
model = DummyClassifier

In [41]:
result = my_GridSearchCV(model, parametrs, features, train['toxic'])

In [42]:
grid_report = grid_report.append(result)

In [43]:
grid_report

,model,rmse,params
0,DummyClassifier,-0.172010,{'strategy': 'uniform'}
0,DummyClassifier,-0.169123,{'strategy': 'uniform'}
0,DummyClassifier,0.170937,{'strategy': 'uniform'}


# 3. Выводы

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны